In [1]:
import numpy as np

In [2]:
import sqlexperiment
# import experiment

In [3]:
from sqlexperiment import ExperimentLog
from sqlexperiment import pseudo, extract

In [4]:
import datetime
datetime_format = "%Y-%m-%d_%H:%M:%S"
def datetime_now():
    return datetime.datetime.now().strftime(datetime_format)

The purpose of this notebook is to produce the data and run the analysis that is described in http://ademos.people.uic.edu/Chapter21.html.


# experiment

We conduct a within-subject experiment with N factors, each of which have M levels.

In [10]:
factors = [('F0', ['a', 'b']), ('F1', ['x', 'y'])]
levels = [np.array(['a', 'b']), np.array(['x', 'y'])]

In [14]:
e = ExperimentLog(":memory:", ntp_sync=False)

In [15]:
if e.meta.stage == 'init':
    # needed?
    e.create('SESSION', 'data', description='')
    e.create('USER', 'user', description='user session')

    for factor in factors:
        e.create("SESSION", factor[0], stype='factor')
        for level in factor[1]:
            e.create("SESSION", level, stype='level')

    e.meta.stage='setup'

# design

We will save design of the experiment in file or in the log, which will be traversed in order to conduct the experiment.
This step can be assisted with other packages, in order for example to have a Latin square design.

2 cases: we have a fixed number of users, or we will add some as we go.

In [16]:
import hashlib

def seed_rng_with_username(username):
    m = hashlib.md5(username)
    seed = int(m.hexdigest()[:8], 16)
    np.random.seed(seed)
    
def shuffle_(data):
    t = data[:]
    np.random.shuffle(t)
    np.random.shuffle(t.T)
    return t

def cartesian_product(*arrays):
    la = len(arrays)
    dtype = np.find_common_type([a.dtype for a in arrays], [])
    arr = np.empty([len(a) for a in arrays] + [la], dtype=dtype)
    for i, a in enumerate(np.ix_(*arrays)):
        arr[...,i] = a
    return arr.reshape(-1, la)

In [17]:
n_users = 3

design = []

for user in range(n_users):
    username = pseudo.get_pseudo()

    cp = cartesian_product(*levels)
    shuffle_(cp)
    cdt = cp[:,0] + cp[:,1]
    design.append((username, cdt.tolist()))

In [18]:
# save the users
for r in design:
    username = r[0]
#     meta = extract.meta_dataframe(e.cursor)
#     is_new_user = not meta['USER']['name'].isin([username]).any()
#     if is_new_user: e.create("USER", name=username)
    e.create("USER", name=username)

# run

In [19]:
meta = extract.meta_dataframe(e.cursor)

In [20]:
sessions = extract.dump_sessions_dataframe(e.cursor)
meta = extract.meta_dataframe(e.cursor)

In [21]:
sessions

,complete,description,end_time,json,last_time,log_count,name,parent,path,random_seed,start_time,subcount,test_run,valid
id,,,,,,,,,,,,,,
1,None,None,None,None,None,0,[ROOT],None,/,None,1.501237e+09,0,None,None


In [22]:
meta['USER']

,bound,data,description,name,type
0,[],None,user session,user,
1,[],None,,WIGIN-EZOPA,
2,[],None,,FADIS-EFIFA,
3,[],None,,CUCAB-UBIVE,


In [23]:
meta['SESSION']

,bound,data,description,name,type
0,[],None,,data,
1,[],None,,F0,factor
2,[],None,,a,level
3,[],None,,b,level
4,[],None,,F1,factor
5,[],None,,x,level
6,[],None,,y,level


In [24]:
# run the full experiment
for run in design:
    
    # select the user
    user = run[0]

    # get a unique session
    session_now = user + '_' + datetime_now()
    e.create('SESSION', session_now, data={'user':user}, stype='exprun')
    e.enter("data", session = session_now)
    # bind after the first enter
    e.bind('USER', user)


    # iterate through the treatments
    for t in run[1]:
        f0, f1 = t
        e.enter(f0, session = f0)
        e.enter(f1, session = f1)
        
        for i in range(3):
            e.enter()
            data = np.random.random_sample(1)[0]

            # tweak the data to show effect and interaction
            e.log("data", data={'d':data, 'F0':f0, 'F1':f1, 'level':t, 'user': user})
            e.leave()

        e.leave()
        e.leave()
    e.leave()

07-28 11:17 [WARNI]  No stream data registered; creating a new blank entry


In [29]:
flatdf = extract.dump_flat_dataframe(e.cursor)
flatdf['data'].head(10)

,d,F0,F1,level,user,t,valid,session_valid,path,session
0,0.996347,a,y,ay,WIGIN-EZOPA,1.501237e+09,1,1,/data/a/y/0/,5
1,0.304371,a,y,ay,WIGIN-EZOPA,1.501237e+09,1,1,/data/a/y/1/,6
2,0.723445,a,y,ay,WIGIN-EZOPA,1.501237e+09,1,1,/data/a/y/2/,7
3,0.273164,a,x,ax,WIGIN-EZOPA,1.501237e+09,1,1,/data/a/x/0/,10
4,0.729677,a,x,ax,WIGIN-EZOPA,1.501237e+09,1,1,/data/a/x/1/,11
5,0.002068,a,x,ax,WIGIN-EZOPA,1.501237e+09,1,1,/data/a/x/2/,12
6,0.101160,b,x,bx,WIGIN-EZOPA,1.501237e+09,1,1,/data/b/x/0/,15
7,0.094407,b,x,bx,WIGIN-EZOPA,1.501237e+09,1,1,/data/b/x/1/,16
8,0.167374,b,x,bx,WIGIN-EZOPA,1.501237e+09,1,1,/data/b/x/2/,17
9,0.402104,b,y,by,WIGIN-EZOPA,1.501237e+09,1,1,/data/b/y/0/,20


In [28]:
sessions = extract.dump_sessions_dataframe(e.cursor)
sessions.head(10)

,complete,description,end_time,json,last_time,log_count,name,parent,path,random_seed,start_time,subcount,test_run,valid
id,,,,,,,,,,,,,,
1,NaN,None,NaN,None,NaN,0,[ROOT],NaN,/,NaN,1.501237e+09,0,NaN,NaN
2,1.0,,1.501237e+09,None,NaN,0,data,1.0,/data/,1.501237e+12,1.501237e+09,0,0.0,1.0
3,1.0,,1.501237e+09,None,NaN,0,a,2.0,/data/a/,3.002474e+12,1.501237e+09,0,0.0,1.0
4,1.0,,1.501237e+09,None,NaN,0,y,3.0,/data/a/y/,4.503711e+12,1.501237e+09,3,0.0,1.0
5,1.0,,1.501237e+09,None,1.501237e+09,1,0,4.0,/data/a/y/0/,6.004948e+12,1.501237e+09,0,0.0,1.0
6,1.0,,1.501237e+09,None,1.501237e+09,1,1,4.0,/data/a/y/1/,6.004948e+12,1.501237e+09,0,0.0,1.0
7,1.0,,1.501237e+09,None,1.501237e+09,1,2,4.0,/data/a/y/2/,6.004948e+12,1.501237e+09,0,0.0,1.0
8,1.0,,1.501237e+09,None,NaN,0,a,2.0,/data/a/,3.002474e+12,1.501237e+09,0,0.0,1.0
9,1.0,,1.501237e+09,None,NaN,0,x,8.0,/data/a/x/,1.200990e+13,1.501237e+09,3,0.0,1.0


In [27]:
meta = extract.meta_dataframe(e.cursor)
meta['SESSION']

,bound,data,description,name,type
0,[],None,,data,
1,[],None,,F0,factor
2,"[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 30, 31, 32, ...",None,,a,level
3,"[13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 25, 2...",None,,b,level
4,[],None,,F1,factor
5,"[9, 10, 11, 12, 14, 15, 16, 17, 24, 25, 26, 27...",None,,x,level
6,"[4, 5, 6, 7, 19, 20, 21, 22, 34, 35, 36, 37, 3...",None,,y,level
7,"[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1...",{'user': 'WIGIN-EZOPA'},,WIGIN-EZOPA_2017-07-28_11:17:09,exprun
8,"[23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 3...",{'user': 'FADIS-EFIFA'},,FADIS-EFIFA_2017-07-28_11:17:09,exprun
9,"[44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 5...",{'user': 'CUCAB-UBIVE'},,CUCAB-UBIVE_2017-07-28_11:17:09,exprun


# analysis